In [ ]:
import sys
import os

import geopandas as gpd


sys.path.insert(0, "/home/wb411133/Code/gostrocks/src")

In [ ]:
input_folder = "/home/wb411133/projects/URB_slum_mapping/data/"
boundary_folder = os.path.join(input_folder, "boundary")
wb_footprints = os.path.join(input_folder, "WB_outlines_footgeo.geojson")

in_shps = []
for root, folder, files in os.walk(boundary_folder):
    for f in files:
        if f.endswith(".shp"):
            in_shps.append(os.path.join(root, f))
in_wb = gpd.read_file(wb_footprints)
in_wb["geometry"] = in_wb["geometry"].apply(lambda x: x.buffer(0))

In [ ]:
in_shps

In [ ]:
good_res = {}
for cur_shp in in_shps:
    inD = gpd.read_file(cur_shp)
    if inD.crs != in_wb.crs:
        inD = inD.to_crs(in_wb.crs)
    if in_wb.unary_union.intersects(inD.unary_union):
        sel_wb = in_wb.loc[in_wb.intersects(inD.unary_union)]
        good_res[cur_shp] = sel_wb.copy()

In [ ]:
all_size = 0
for shp, images in good_res.items():
    city = os.path.basename(os.path.dirname(shp))
    for idx, row in images.iterrows():
        print(f"{row['location']}")
        all_size += float(row["zippedSize"])

In [ ]:
good_res.keys()

In [ ]:
all_size / 1024 / 1024 / 1024